In [1]:
import pandas as pd
from colorthief import ColorThief
import io
import urllib
import os
import re
import colorgram
import numpy as np

In [2]:
# import raw data
artist = pd.read_csv('../raw_data/artist_data.csv')
artwork = pd.read_csv('../raw_data/artwork_data.csv', dtype={'creditLine': str, 'height': str, 'year': str})

In [3]:
# graph 1: # artists artwork, gender in Tate collection
# maybe for future: age @ first Tate acquisition?

# merge artist and artwork on artist id
artists_artwork = artist.merge(artwork, 'left', left_on = 'id', right_on = 'artistId')
aa_counts = artists_artwork.groupby('id_x')['name'].count()
aa_counts = aa_counts.to_frame().rename(columns = {'name': 'artwork_count'}).reset_index()

# merge in gender
aa_counts = aa_counts.merge(artist, left_on='id_x', right_on='id')
aa_counts = aa_counts[['id', 'name', 'artwork_count', 'gender']]

# # save to csv
# aa_counts.to_csv('../cleaned_data/aa_counts.csv')

In [4]:
# graph 1: artists artwork (1 each)
def get_pics1(row):
    ''' 
    Takes row of dataframe & downloads image from url
    '''
    id, _, _, _, _, url = row
    print(id)
    if type(url) == str:
        urllib.request.urlretrieve(url, f'/Users/carolynliu/CAPP30239_FA22/final_project/pics1/{id}.jpg')

In [5]:
# graph 1 get pics
df = pd.read_csv('../raw_data/the-tate-collection.csv', sep=';')
df = df[['artistId', 'thumbnailUrl']]
df.drop_duplicates('artistId', inplace=True)

df1 = aa_counts.merge(df, left_on='id', right_on='artistId')
# df1.apply(lambda x: get_pics1(x), axis = 1)

In [6]:
df1

,id,name,artwork_count,gender,artistId,thumbnailUrl
0,0,"Abbey, Edwin Austin",6,Male,0,https://public.opendatasoft.com/api/v2/catalog...
1,1,"Abbott, Lemuel Francis",2,Male,1,https://public.opendatasoft.com/api/v2/catalog...
2,2,"Agasse, Jacques Laurent",2,Male,2,https://public.opendatasoft.com/api/v2/catalog...
3,3,"Alken, Henry Thomas",5,Male,3,https://public.opendatasoft.com/api/v2/catalog...
4,4,"Allan, Sir William",1,Male,4,https://public.opendatasoft.com/api/v2/catalog...
...,...,...,...,...,...,...
3333,17412,"Whitten, Jack",1,Male,17412,NaN
3334,17414,"Fukushima, Hideko",1,Female,17414,https://public.opendatasoft.com/api/v2/catalog...
3335,17422,"Wylie, Rose",1,Female,17422,NaN
3336,17450,"Crozier, William",1,Male,17450,NaN


In [7]:
df1['img'] = df1.apply(lambda x: '' if type(x['thumbnailUrl'])==float else f'../pics1/{x["id"]}.jpg', axis = 1)
df1.drop(['artistId', 'thumbnailUrl'], axis = 1, inplace=True)
df1['parent'] = 'parent'
df1['color'] = df1.apply(lambda x: '#FF7133' if x['gender']=='Female' else '#33B5FF', axis = 1)
df1.to_csv('../cleaned_data/aa_counts_url.csv')

In [ ]:
df1

In [8]:
# graph 2: # Tate acquisition over time & medium?

acquisitions = artwork.groupby(['acquisitionYear'])['acquisitionYear'].count()
acquisitions = acquisitions.to_frame().rename(columns = {'acquisitionYear': 'ac_count'}).reset_index()

# most popular medium by acquisition year
medium = artwork.groupby(['acquisitionYear', 'medium'])['medium'].count()
medium = medium.to_frame().rename(columns = {'medium': 'med_count'}).reset_index()
medium = medium.groupby('acquisitionYear', group_keys=False).apply(lambda x: x.loc[x.med_count.idxmax()]).reset_index(drop=True)

# merge acquisitions and medium
ac_med = acquisitions.merge(medium)
ac_med['year'] = ac_med['acquisitionYear'].astype(int).astype(str)
#save to csv
ac_med.to_csv('../cleaned_data/ac_med.csv')

In [ ]:
# graph 3 - colors
# keep only watercolors
wc = artwork[artwork['medium'].str.contains("watercolour", case = False, na=False)]
wc.to_csv('../raw_data/watercolours.csv')

In [ ]:
# data for artwork thumbnails from opendatasoft - https://public.opendatasoft.com/explore/dataset/the-tate-collection/table/
df = pd.read_csv('../raw_data/the-tate-collection-3.csv', sep=';')
df = df[['accession_number', 'thumbnailUrl']]

In [ ]:
#merge w/ watercolors
wc1 = wc.merge(df, on = 'accession_number')
wc2 = wc1[['accession_number', 'thumbnailUrl_y']]
wc2.dropna(inplace=True)

In [ ]:
url = 'https://public.opendatasoft.com/api/v2/catalog/datasets/the-tate-collection/files/3fa7321cc3eac4b92b2409fcf5c207a4'
urllib.request.urlretrieve(url, f'/Users/carolynliu/CAPP30239_FA22/final_project/pics/test.jpg')

In [ ]:
def get_pics(row):
    ''' 
    Takes row of dataframe & downloads image from url
    '''
    num, url = row
    print(num)
    urllib.request.urlretrieve(url, f'/Users/carolynliu/CAPP30239_FA22/final_project/pics/{num}.jpg')

In [ ]:
wc2.apply(lambda x: get_pics(x), axis = 1)
# got thru D01659

In [ ]:
# take out rows already have images for & download images
wc3 = wc2.loc[wc2.loc[wc2['accession_number'] == 'D01660'].index[0]: ,]
wc3.apply(lambda x: get_pics(x), axis = 1)

In [ ]:
# Get dominant (color thief) color for watercolor artworks
dir = '/Users/carolynliu/CAPP30239_FA22/final_project/pics'
colors = {}
for filename in os.listdir(dir):
    exp = re.compile(".jpg")
    if re.search(exp, filename):
        f = os.path.join(dir, filename)
        color_thief = ColorThief(f)
        # get the dominant color
        dominant_color = color_thief.get_color(quality=1)
        #color_thief.get_palette(color_count=6)
        num = filename.split('.')[0]
        colors[num] = dominant_color


In [ ]:
# COLOR THIEF make into pd df & merge w/ watercolors data
colors_df = pd.DataFrame.from_dict(colors, orient = 'index').reset_index()
colors_df.rename(columns={'index': 'accession_number', 0:'r', 1:'g', 2:'b'}, inplace=True)

watercolors = wc.merge(colors_df)
watercolors = watercolors[['accession_number', 'year', 'r', 'g', 'b' ]]
watercolors.to_csv('../cleaned_data/watercolors.csv')

In [ ]:
# colorgram! 
# Get (colorgram) color for watercolor artworks
dir = '/Users/carolynliu/CAPP30239_FA22/final_project/pics'
colors = {}
for filename in os.listdir(dir):
    exp = re.compile(".jpg")
    if re.search(exp, filename):
        f = os.path.join(dir, filename)
        color_thief = ColorThief(f)
        # get 3 colors
        c = colorgram.extract(f, 3)
        num = filename.split('.')[0]
        colors[num] = c

# make into pd df & merge w/ watercolors data
colors_df = pd.DataFrame.from_dict(colors, orient = 'index').reset_index()

# split columns to make data easier to use
cols = ['rgb', 'prop']
for i in range(3):
    rgb = 'rgb' + str(i)
    prop = 'prop' + str(i)
    colors_df[rgb] = colors_df.apply(lambda row: tuple(row[i].rgb), axis=1)
    colors_df[prop] = colors_df.apply(lambda row: row[i].proportion, axis=1)

colors_df.rename(columns={'index': 'accession_number'}, inplace=True)

watercolors_cg = wc.merge(colors_df)
watercolors_cg = watercolors_cg[['accession_number', 'year', 'rgb0', 'prop0',	'rgb1',	'prop1', 'rgb2', 'prop2']]
watercolors_cg.to_csv('../cleaned_data/watercolors_cg.csv')